In [ ]:
!pip install gradio transformers torch torchvision
!pip install Pillow opencv-python matplotlib pandas numpy
!pip install easyocr pdf2image python-multipart
!pip install sacremoses

print("✅ All packages installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 12.3 MB/s eta 0:00:00
✅ All packages installed!


In [ ]:
from google.colab import files
import pandas as pd
import os

print("📤 UPLOAD YOUR MEDICAL RULES CSV")
print("=" * 50)
print("Please upload the 'medical_rules_template.csv' file you downloaded earlier")
print("=" * 50)

# Upload the CSV file
uploaded = files.upload()

# Check if CSV was uploaded
if uploaded:
    csv_filename = list(uploaded.keys())[0]
    print(f"✅ Uploaded: {csv_filename}")

    # Load the CSV with proper handling for commas in fields
    try:
        medical_rules_df = pd.read_csv(csv_filename, quotechar='"', skipinitialspace=True)
        print(f"📊 Successfully loaded {len(medical_rules_df)} medical rules")

        # Display sample
        print("\n📋 Sample of your medical rules:")
        print(medical_rules_df.head(3).to_string(index=False))

        # Check column structure
        print(f"\n📝 Columns in your CSV: {list(medical_rules_df.columns)}")

        # Check if we have gender-specific columns
        has_gender_specific = 'normal_min_male' in medical_rules_df.columns and 'normal_min_female' in medical_rules_df.columns

        if has_gender_specific:
            print("✅ Gender-specific ranges detected")
        else:
            print("ℹ️ Using standard ranges (no gender-specific data)")

    except Exception as e:
        print(f"❌ Error loading CSV: {e}")
        print("Trying alternative loading method...")

        # Try reading with different parameters
        try:
            medical_rules_df = pd.read_csv(csv_filename, encoding='utf-8', on_bad_lines='skip')
            print(f"📊 Alternative load successful: {len(medical_rules_df)} medical rules")
        except Exception as e2:
            print(f"❌ All loading methods failed: {e2}")
            raise Exception("Could not load CSV file. Please check the format.")

else:
    print("❌ No CSV uploaded. Please upload your medical rules CSV file.")
    raise Exception("No medical rules CSV file uploaded")

print("✅ Medical rules loaded successfully!")

📤 UPLOAD YOUR MEDICAL RULES CSV
Please upload the 'medical_rules_template.csv' file you downloaded earlier


Saving medical_rules_template.csv to medical_rules_template.csv
✅ Uploaded: medical_rules_template.csv
❌ Error loading CSV: Error tokenizing data. C error: Expected 14 fields in line 23, saw 16

Trying alternative loading method...
📊 Alternative load successful: 38 medical rules
✅ Medical rules loaded successfully!


In [ ]:
# Convert to dictionary for use in analysis
MEDICAL_REFERENCE = {}

for _, row in medical_rules_df.iterrows():
    test_info = {
        'display_name': row['display_name'],
        'unit': row['unit'],
        'normal_male': [row['normal_min_male'], row['normal_max_male']],
        'normal_female': [row['normal_min_female'], row['normal_max_female']],
        'low_severity': row['low_severity'],
        'high_severity': row['high_severity'],
        'low_explanation': row['low_explanation'],
        'high_explanation': row['high_explanation'],
        'search_patterns': row['search_patterns'].split('|') if isinstance(row['search_patterns'], str) else row['search_patterns'],
        'description': row['description'],
        'category': row['category'],
        'has_gender_specific': True
    }

    MEDICAL_REFERENCE[row['test_name']] = test_info

print(f"🎯 Medical reference loaded: {len(MEDICAL_REFERENCE)} tests")
print("📋 Available tests:", list(MEDICAL_REFERENCE.keys()))

# Check if we have gender-specific data
has_gender_data = any(info.get('has_gender_specific', False) for info in MEDICAL_REFERENCE.values())
print(f"🎭 Gender-specific data: {'✅ Available' if has_gender_data else '❌ Not available'}")

🎯 Medical reference loaded: 38 tests
📋 Available tests: ['hemoglobin', 'rbc', 'wbc', 'platelets', 'hct', 'mcv', 'mch', 'mchc', 'neutrophils', 'lymphocytes', 'monocytes', 'eosinophils', 'basophils', 'rdw', 'glucose', 'sodium', 'potassium', 'chloride', 'bicarbonate', 'bun', 'creatinine', 'cholesterol_total', 'ldl', 'hdl', 'triglycerides', 'alt', 'bilirubin_total', 'albumin', 'total_protein', 'tsh', 'free_t4', 'free_t3', 'esr', 'crp', 'vitamin_d', 'vitamin_b12', 'ferritin', 'iron']
🎭 Gender-specific data: ✅ Available


In [ ]:
import torch
from transformers import pipeline
import easyocr
import re
import numpy as np

print("🔄 Loading FAST Medical AI Models...")

# 1. Load lightweight models for speed
try:
    # Use a smaller model for text generation
    medical_explainer = pipeline(
        "text2text-generation",
        model="google/flan-t5-small",
        max_length=150,
        torch_dtype=torch.float32
    )
    print("✅ Fast text generator loaded successfully!")
except Exception as e:
    print(f"❌ Fast generator failed: {e}")
    medical_explainer = None

# 2. Load OCR (keep this - it's necessary)
try:
    reader = easyocr.Reader(['en'], gpu=False)
    print("✅ EasyOCR loaded successfully!")
except Exception as e:
    print(f"❌ EasyOCR loading failed: {e}")
    reader = None

print("🎯 FAST models loaded! Ready for instant analysis.")
print("📊 Models available: Text Generator + EasyOCR")

🔄 Loading FAST Medical AI Models...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


✅ Fast text generator loaded successfully!
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete✅ EasyOCR loaded successfully!
🎯 FAST models loaded! Ready for instant analysis.
📊 Models available: Text Generator + EasyOCR


In [ ]:
import cv2
import pandas as pd
import re
from PIL import Image
import numpy as np

def enhanced_ocr_processing(lab_image):
    """ULTRA-ENHANCED OCR processing specifically for lab reports"""
    try:
        img_cv = np.array(lab_image.convert('RGB'))
        img_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2BGR)
        gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

        # Multiple aggressive preprocessing techniques
        processed_images = []

        # 1. Original grayscale
        processed_images.append(gray)

        # 2. High contrast CLAHE
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        enhanced_clahe = clahe.apply(gray)
        processed_images.append(enhanced_clahe)

        # 3. Morphological operations to enhance text
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
        morph = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)
        processed_images.append(morph)

        # 4. Adaptive threshold
        adaptive_thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                              cv2.THRESH_BINARY, 11, 2)
        processed_images.append(adaptive_thresh)

        all_text = ""
        best_text = ""
        max_text_length = 0

        # Try OCR on all processed versions
        for i, proc_img in enumerate(processed_images):
            print(f"🔍 OCR Pass {i+1}...")
            try:
                # Try with different OCR parameters
                results = reader.readtext(proc_img, detail=1, paragraph=False,
                                        width_ths=0.7, height_ths=0.7)
                text_chunk = "\n".join([result[1] for result in results])

                if len(text_chunk) > max_text_length:
                    max_text_length = len(text_chunk)
                    best_text = text_chunk

                all_text += f"\n--- PASS {i+1} ---\n{text_chunk}"
                print(f"   Found {len(results)} text elements")

            except Exception as e:
                print(f"   OCR pass {i+1} failed: {e}")
                continue

        print(f"📄 Best OCR text length: {len(best_text)} characters")
        return best_text, all_text

    except Exception as e:
        print(f"❌ OCR processing failed: {e}")
        return "", ""

def extract_all_lab_values(extracted_text):
    """COMPREHENSIVE extraction for ALL lab tests from uploaded CSV"""
    lab_values = {}

    # Build test patterns dynamically from the uploaded CSV
    test_patterns = {}

    for test_name, test_info in MEDICAL_REFERENCE.items():
        patterns = []
        search_terms = test_info['search_patterns']

        # Create patterns for each search term
        for term in search_terms:
            # Pattern 1: Test name followed by value and unit
            patterns.append(r'(?i)' + re.escape(term) + r'[\s:\-]*([\d.]+)\s*(?:' + re.escape(test_info['unit']) + r')?')
            # Pattern 2: Test name followed by value (more flexible)
            patterns.append(r'(?i)' + re.escape(term) + r'[\s:\-]*([\d.]+)')
            # Pattern 3: Just the value near the test name
            patterns.append(r'(?i)' + re.escape(term) + r'.*?([\d.]+)')

        test_patterns[test_name] = patterns

    # First pass: Direct pattern matching
    for test_name, patterns in test_patterns.items():
        for pattern in patterns:
            matches = re.finditer(pattern, extracted_text)
            for match in matches:
                try:
                    value_str = match.group(1)
                    value = float(value_str)

                    if is_reasonable_value(test_name, value) and test_name not in lab_values:
                        lab_values[test_name] = value
                        print(f"✅ Found {test_name}: {value}")
                        break  # Move to next test after first valid match
                except (ValueError, IndexError) as e:
                    continue

    # Second pass: Table parsing - look for test names and nearby values
    lines = extracted_text.split('\n')

    for i, line in enumerate(lines):
        line_clean = line.strip()

        for test_name, test_info in MEDICAL_REFERENCE.items():
            if test_name in lab_values:  # Skip if already found
                continue

            # Check if any search term is in this line
            search_terms = test_info['search_patterns']
            for term in search_terms:
                if term.upper() in line_clean.upper():
                    # Look for numbers in current line
                    numbers = re.findall(r'(\d+\.?\d*)', line_clean)
                    for num in numbers:
                        try:
                            value = float(num)
                            if is_reasonable_value(test_name, value) and test_name not in lab_values:
                                lab_values[test_name] = value
                                print(f"✅ Table match - {test_name}: {value}")
                                break
                        except ValueError:
                            pass

                    # Look in next 3 lines for values
                    for j in range(1, 4):
                        if i + j < len(lines):
                            next_line = lines[i + j]
                            numbers = re.findall(r'(\d+\.?\d*)', next_line)
                            for num in numbers:
                                try:
                                    value = float(num)
                                    if is_reasonable_value(test_name, value) and test_name not in lab_values:
                                        lab_values[test_name] = value
                                        print(f"✅ Nearby line - {test_name}: {value}")
                                        break
                                except ValueError:
                                    pass

    return lab_values

def analyze_lab_report_image(lab_image):
    """
    Analyze lab report from image
    """
    try:
        print("🔍 Starting ULTRA-IMPROVED analysis from image...")

        if reader is None:
            return error_response("OCR engine not available")

        # Enhanced OCR processing
        best_text, all_ocr_passes = enhanced_ocr_processing(lab_image)

        if not best_text or len(best_text.strip()) < 20:
            return error_response("No readable text found in image")

        print("📋 Sample of extracted text:")
        print(best_text[:500] + "..." if len(best_text) > 500 else best_text)

        # Comprehensive value extraction
        lab_values = extract_all_lab_values(best_text)

        print(f"🎯 FINAL EXTRACTION: {len(lab_values)} values found")
        for test, value in lab_values.items():
            print(f"   {test.upper()}: {value}")

        if not lab_values:
            # Last resort: try all OCR passes
            lab_values = extract_all_lab_values(all_ocr_passes)
            print(f"🔄 SECOND ATTEMPT: {len(lab_values)} values found")

        if not lab_values:
            return error_response("No lab values detected. Image may be unclear or format not recognized.")

        # Perform medical analysis
        return perform_medical_analysis(lab_values, best_text)

    except Exception as e:
        print(f"❌ Analysis failed: {e}")
        import traceback
        traceback.print_exc()
        return error_response(f"System error: {str(e)}")

def analyze_lab_report_text(text_input, gender="male"):
    """
    Analyze lab report from direct text input
    """
    try:
        print("🔍 Starting analysis from text input...")

        if not text_input or len(text_input.strip()) < 10:
            return error_response("Please provide meaningful lab report text")

        # Comprehensive value extraction
        lab_values = extract_all_lab_values(text_input)

        print(f"🎯 TEXT EXTRACTION: {len(lab_values)} values found")
        for test, value in lab_values.items():
            print(f"   {test.upper()}: {value}")

        if not lab_values:
            return error_response("No lab values detected in the provided text.")

        # Perform medical analysis with gender
        return perform_medical_analysis(lab_values, text_input, gender)

    except Exception as e:
        print(f"❌ Text analysis failed: {e}")
        import traceback
        traceback.print_exc()
        return error_response(f"System error: {str(e)}")

def perform_medical_analysis(lab_values, extracted_text, gender="male"):
    """Perform medical analysis on extracted values"""
    models_used = []
    abnormal_count = 0

    analysis_results = []
    for test_name, value in lab_values.items():
        if test_name in MEDICAL_REFERENCE:
            ref = MEDICAL_REFERENCE[test_name]
            unit = ref["unit"]

            # Get normal range based on gender
            if gender.lower() == 'female':
                normal_min, normal_max = ref["normal_female"]
            else:
                normal_min, normal_max = ref["normal_male"]

            # Handle special cases for percentages
            if test_name in ['neutrophils', 'lymphocytes', 'monocytes', 'eosinophils', 'basophils']:
                # These are percentages, handle decimal points
                if value < 1 and value > 0:  # If value is 0.53 instead of 53
                    value = value * 100

            if value < normal_min:
                status = "LOW"
                explanation = ref["low_explanation"]
                severity = ref["low_severity"]
                color = "#ff6b6b"
                emoji = "🔻"
                abnormal_count += 1
            elif value > normal_max:
                status = "HIGH"
                explanation = ref["high_explanation"]
                severity = ref["high_severity"]
                color = "#ff6b6b"
                emoji = "🔺"
                abnormal_count += 1
            else:
                status = "NORMAL"
                explanation = "Within healthy range"
                severity = "LOW"
                color = "#51cf66"
                emoji = "✅"

            analysis_results.append({
                "test": ref["display_name"],
                "value": f"{value} {unit}",
                "status": status,
                "severity": severity,
                "explanation": explanation,
                "normal_range": f"{normal_min}-{normal_max} {unit}",
                "color": color,
                "emoji": emoji,
                "description": ref.get('description', ''),
                "category": ref.get('category', 'General')
            })

    # Generate intelligent summary
    summary, overall_risk = generate_intelligent_summary(analysis_results, abnormal_count, lab_values)

    # Try AI summary if available
    if medical_explainer and lab_values:
        try:
            values_str = ", ".join([f"{MEDICAL_REFERENCE[k]['display_name']}: {v}" for k, v in lab_values.items()])
            ai_prompt = f"Provide a brief medical interpretation of these lab results: {values_str}"
            ai_response = medical_explainer(ai_prompt, max_length=120)
            summary = ai_response[0]['generated_text']
            models_used.append("AI")
            print("✅ AI summary generated")
        except Exception as e:
            print(f"❌ AI summary failed: {e}")

    return {
        "analysis": analysis_results,
        "summary": summary,
        "overall_risk": overall_risk,
        "models_used": models_used,
        "disclaimer": "⚠️ This is not a medical diagnosis. Please consult a licensed doctor for confirmation.",
        "values_found": len(lab_values),
        "abnormal_count": abnormal_count,
        "raw_text_sample": extracted_text[:300] + "..." if len(extracted_text) > 300 else extracted_text
    }

def generate_intelligent_summary(analysis_results, abnormal_count, lab_values):
    """Generate intelligent summary based on specific findings"""
    if abnormal_count == 0:
        return "All lab values are within normal ranges. Continue regular health monitoring.", "LOW"

    # Check for specific patterns
    abnormal_tests = [item for item in analysis_results if item["status"] != "NORMAL"]
    abnormal_names = [item["test"] for item in abnormal_tests]

    # Check for anemia pattern
    if all(key in lab_values for key in ['mcv', 'mch', 'mchc']):
        if lab_values['mcv'] < 80 and lab_values['mch'] < 27 and lab_values['mchc'] < 32:
            return "Pattern suggests microcytic hypochromic anemia. Please consult your doctor.", "HIGH"

    # Check for infection pattern
    if 'wbc' in lab_values and lab_values['wbc'] > 11:
        return "Elevated white blood cells may indicate infection or inflammation.", "MODERATE"

    # Check for kidney issues
    if 'creatinine' in lab_values and lab_values['creatinine'] > 1.3:
        return "Elevated creatinine may indicate kidney dysfunction.", "HIGH"

    if abnormal_count == 1:
        test_name = abnormal_tests[0]['test']
        return f"One abnormal value found ({test_name}). Discuss with your healthcare provider.", "MODERATE"
    elif abnormal_count <= 3:
        return f"Found {abnormal_count} abnormal values. Recommended to consult your doctor.", "MODERATE"
    else:
        return f"Multiple abnormalities detected ({abnormal_count}). Please consult your doctor promptly.", "HIGH"

def error_response(message):
    """Return standardized error response"""
    return {
        "analysis": [],
        "summary": message,
        "overall_risk": "ERROR",
        "models_used": [],
        "disclaimer": "Please try again with a clearer image or different format.",
        "values_found": 0,
        "abnormal_count": 0,
        "raw_text_sample": ""
    }

def is_reasonable_value(test_name, value):
    """Validate if extracted value is medically reasonable"""
    reasonable_ranges = {
        'hemoglobin': (5, 20), 'rbc': (2, 8), 'wbc': (1, 50), 'platelets': (50, 600),
        'hct': (20, 60), 'mcv': (50, 120), 'mch': (15, 40), 'mchc': (25, 38),
        'neutrophils': (0, 100), 'lymphocytes': (0, 100), 'monocytes': (0, 100),
        'eosinophils': (0, 100), 'basophils': (0, 100), 'rdw': (5, 25),
        'glucose': (30, 500), 'sodium': (100, 160), 'potassium': (2, 7),
        'creatinine': (0.1, 10), 'calcium': (5, 15), 'cholesterol_total': (50, 400),
        'ldl': (10, 300), 'hdl': (10, 100), 'triglycerides': (30, 500),
        'alt': (1, 200), 'ast': (1, 200), 'alp': (10, 300), 'bilirubin_total': (0.1, 10),
        'albumin': (1, 8), 'total_protein': (3, 12), 'tsh': (0.1, 20),
        'free_t4': (0.1, 5), 'free_t3': (1, 10), 'esr': (0, 100), 'crp': (0, 50),
        'vitamin_d': (5, 200), 'vitamin_b12': (50, 2000), 'ferritin': (5, 1000), 'iron': (10, 300)
    }

    if test_name in reasonable_ranges:
        min_val, max_val = reasonable_ranges[test_name]
        return min_val <= value <= max_val
    return True

print("✅ ULTRA-IMPROVED analysis functions ready!")
print("🎯 Now supports both IMAGE UPLOAD and TEXT INPUT!")

✅ ULTRA-IMPROVED analysis functions ready!
🎯 Now supports both IMAGE UPLOAD and TEXT INPUT!


In [ ]:
import gradio as gr

def process_lab_report(image_input, text_input, input_type, gender):
    """Process lab report from either image or text"""
    print("=" * 50)
    print(f"🔄 Processing {input_type} input...")

    if input_type == "image":
        if image_input is None:
            return error_response("Please upload an image first")
        result = analyze_lab_report_image(image_input)
    else:  # text
        if not text_input or not text_input.strip():
            return error_response("Please enter lab report text")
        result = analyze_lab_report_text(text_input, gender)

    print("✅ Analysis completed!")
    print(f"📊 Found {result['values_found']} values, {result['abnormal_count']} abnormal")
    print("=" * 50)
    return format_results_html(result)

def format_results_html(result):
    """Format analysis results as HTML"""
    # Handle errors or no values found
    if result['values_found'] == 0:
        error_html = f"""
        <div style='padding: 25px; background: #ffeaa7; border-radius: 12px; border-left: 5px solid #fdcb6e;'>
            <h3 style='color: #e17055; margin-top: 0;'>❌ Analysis Results</h3>
            <p><strong>{result['summary']}</strong></p>

            <div style='background: #fff3cd; padding: 15px; border-radius: 8px; margin-top: 15px;'>
                <h4 style='margin: 0 0 10px 0; color: #856404;'>💡 Tips for Better Results:</h4>
                <ul style='margin: 0;'>
                    <li>Use high-quality, clear images</li>
                    <li>Ensure the entire table is visible</li>
                    <li>Good lighting and focus</li>
                    <li>Common formats: CBC, Blood Count reports</li>
                    <li>Text should be sharp and not blurry</li>
                </ul>
            </div>
        """

        if result.get('raw_text_sample'):
            error_html += f"""
            <div style='margin-top: 15px; padding: 10px; background: #f8f9fa; border-radius: 5px; font-size: 11px; font-family: monospace;'>
                <strong>OCR Text Sample:</strong><br>{result['raw_text_sample']}
            </div>
            """

        error_html += "</div>"
        return error_html

    # Create analysis cards for ALL found values
    analysis_cards = ""
    for item in result['analysis']:
        analysis_cards += f"""
        <div style='background: {item["color"]}15; padding: 15px; margin: 10px 0; border-radius: 10px; border-left: 4px solid {item["color"]};'>
            <div style='display: flex; justify-content: space-between; align-items: center;'>
                <h4 style='margin: 0; color: #2d3436;'>{item["emoji"]} {item["test"]}</h4>
                <span style='background: {item["color"]}; color: white; padding: 4px 12px; border-radius: 20px; font-size: 12px; font-weight: bold;'>
                    {item["status"]}
                </span>
            </div>
            <div style='margin-top: 10px;'>
                <p style='margin: 5px 0;'><strong>Value:</strong> {item["value"]}</p>
                <p style='margin: 5px 0;'><strong>Normal Range:</strong> {item["normal_range"]}</p>
                <p style='margin: 5px 0;'><strong>Interpretation:</strong> {item["explanation"]}</p>
                <p style='margin: 5px 0;'><strong>Severity:</strong> {item["severity"]}</p>
                <p style='margin: 5px 0; font-size: 12px; color: #666;'><em>{item.get("description", "")}</em></p>
            </div>
        </div>
        """

    # Models used badge
    models_badge = " + ".join(result['models_used']) if result['models_used'] else "Enhanced Rule-Based Analysis"

    # Risk color coding
    risk_color = {
        "LOW": "#51cf66",
        "MODERATE": "#ffa500",
        "HIGH": "#ff6b6b",
        "ERROR": "#6c757d"
    }.get(result['overall_risk'], "#6c757d")

    output_html = f"""
    <div style='padding: 25px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); border-radius: 12px; color: white;'>
        <h2 style='margin-top: 0;'>🩺 Comprehensive Lab Report Analysis</h2>
        <p style='font-size: 16px; margin-bottom: 15px;'><strong>Clinical Summary:</strong> {result['summary']}</p>

        <div style='display: flex; gap: 15px; margin-bottom: 10px; flex-wrap: wrap;'>
            <span style='background: rgba(255,255,255,0.2); padding: 8px 16px; border-radius: 20px; font-size: 14px;'>
                🔬 {result['values_found']} Tests Analyzed
            </span>
            <span style='background: rgba(255,255,255,0.2); padding: 8px 16px; border-radius: 20px; font-size: 14px;'>
                📊 {result['abnormal_count']} Abnormal Findings
            </span>
            <span style='background: rgba(255,255,255,0.2); padding: 8px 16px; border-radius: 20px; font-size: 14px;'>
                🤖 {models_badge}
            </span>
            <span style='background: {risk_color}; padding: 8px 16px; border-radius: 20px; font-size: 14px; font-weight: bold;'>
                📈 {result['overall_risk']} Risk Level
            </span>
        </div>
    </div>

    <div style='padding: 20px; background: #f8f9fa; border-radius: 12px; margin-top: 15px;'>
        <h3 style='color: #2d3436; margin-top: 0;'>📊 Detailed Test Analysis</h3>
        {analysis_cards}

        <div style='background: #d4edda; padding: 15px; border-radius: 8px; margin-top: 20px; border-left: 4px solid #28a745;'>
            <h4 style='margin: 0 0 10px 0; color: #155724;'>✅ Analysis Complete</h4>
            <p style='margin: 0; font-size: 14px;'>Successfully analyzed {result['values_found']} laboratory parameters with {result['abnormal_count']} abnormal findings.</p>
        </div>

        <div style='background: #fff3cd; padding: 15px; border-radius: 8px; margin-top: 15px; border-left: 4px solid #ffc107;'>
            <h4 style='margin: 0 0 10px 0; color: #856404;'>⚠️ Important Medical Disclaimer</h4>
            <p style='margin: 0; font-size: 14px;'>{result['disclaimer']}</p>
        </div>
    </div>
    """

    return output_html

print("✅ ENHANCED Gradio processing function ready!")

✅ ENHANCED Gradio processing function ready!


In [ ]:
# Create and launch the Gradio app
with gr.Blocks(
    theme=gr.themes.Soft(
        primary_hue="blue",
        secondary_hue="slate"
    ),
    title="Medical Lab Report Analyzer - DUAL MODE",
    css="""
    .gradio-container {
        max-width: 1200px !important;
    }
    .container {
        max-width: 1200px;
        margin: 0 auto;
    }
    """
) as demo:

    with gr.Column(elem_classes="container"):
        gr.Markdown("""
        # 🩺 Medical Lab Report Analyzer - DUAL MODE
        **Upload image OR paste text for ACCURATE AI-powered analysis**

        *Now supports both Image Upload and Direct Text Input*
        """)

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### ⚙️ Input Options")

                input_type = gr.Radio(
                    choices=["image", "text"],
                    value="image",
                    label="Choose Input Type",
                    info="Upload image or paste text"
                )

                gender = gr.Radio(
                    choices=["male", "female"],
                    value="male",
                    label="Patient Gender",
                    info="For gender-specific reference ranges"
                )

                with gr.Group(visible=True) as image_input_group:
                    gr.Markdown("#### 📤 Upload Lab Report Image")
                    image_input = gr.Image(
                        type="pil",
                        label="Upload Lab Report Image",
                        sources=["upload"],
                        height=250
                    )

                with gr.Group(visible=False) as text_input_group:
                    gr.Markdown("#### 📝 Paste Lab Report Text")
                    text_input = gr.Textbox(
                        label="Lab Report Text",
                        placeholder="Paste your lab report text here...\nExample: Hb: 14.2 g/dL, WBC: 6.5 x10⁹/L, Platelets: 250 x10⁹/L",
                        lines=8,
                        max_lines=15
                    )

                analyze_btn = gr.Button(
                    "🔍 Analyze Lab Report",
                    variant="primary",
                    size="lg"
                )

                gr.Markdown("""
                ### 💡 How it works:
                1. **Choose** input type (Image or Text)
                2. **Select** patient gender for accurate ranges
                3. **Upload/Paste** your lab data
                4. **Advanced analysis** extracts all values
                5. **Get detailed** medical interpretation

                ### ✅ Supported Tests:
                - Complete Blood Count (CBC)
                - Basic Metabolic Panel
                - Lipid Profile
                - Liver Function Tests
                - Thyroid Function
                - And many more...
                """)

            with gr.Column(scale=2):
                gr.Markdown("### 📊 Analysis Results")
                results_output = gr.HTML(
                    label="Results",
                    value="""<div style='padding: 30px; text-align: center; color: #666; background: #f8f9fa; border-radius: 10px;'>
                        <h3 style='color: #2c5aa0;'>👆 Choose Input Method</h3>
                        <p><strong>Option 1:</strong> Upload lab report image (PNG, JPG, JPEG)</p>
                        <p><strong>Option 2:</strong> Paste lab report text directly</p>
                        <div style='background: #e3f2fd; padding: 15px; border-radius: 8px; margin-top: 15px;'>
                            <strong>🎯 Now with DUAL input mode!</strong><br>
                            Better accuracy for both images and text
                        </div>
                    </div>"""
                )

        # Show/hide input groups based on selection
        def toggle_inputs(input_type):
            if input_type == "image":
                return gr.Group(visible=True), gr.Group(visible=False)
            else:
                return gr.Group(visible=False), gr.Group(visible=True)

        input_type.change(
            fn=toggle_inputs,
            inputs=input_type,
            outputs=[image_input_group, text_input_group]
        )

        # Handle the analysis
        analyze_btn.click(
            fn=process_lab_report,
            inputs=[image_input, text_input, input_type, gender],
            outputs=results_output
        )

        gr.Markdown("---")
        gr.Markdown("""
        **Medical Rules Source:** Your uploaded CSV file | **AI Models:** Enhanced OCR + Pattern Matching | **Processing:** Multi-stage analysis

        *Note: This system provides AI-generated insights and is not a substitute for professional medical advice. Always consult healthcare professionals for medical diagnosis.*
        """)

print("🚀 Launching DUAL MODE Medical AI Assistant...")
print("⏰ Processing time: 3-8 seconds per analysis")
print("🎯 Now supports both IMAGE UPLOAD and TEXT INPUT!")
demo.launch(share=True, debug=True)

🚀 Launching DUAL MODE Medical AI Assistant...
⏰ Processing time: 3-8 seconds per analysis
🎯 Now supports both IMAGE UPLOAD and TEXT INPUT!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e0c6948680a9adbb29.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🔄 Processing image input...
🔍 Starting ULTRA-IMPROVED analysis from image...
🔍 OCR Pass 1...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


   Found 58 text elements
🔍 OCR Pass 2...
   Found 58 text elements
🔍 OCR Pass 3...
   Found 58 text elements
🔍 OCR Pass 4...
   Found 58 text elements
📄 Best OCR text length: 685 characters
📋 Sample of extracted text:
Jusss%sis
DoW LAB
DiAqtetdf
Dfagnoslic Reference & Research Laboratory
MR No:
250830-119204321
Age/Gender:  51Y MMale
Lab No:   0624372344
Referred By:  SELF
Name:
HYDER ALI KHAN
Sample Date: 30-Aug-2025
CP(COMPLETE BLQODPICTURE)
TEST(s)
RESULT(s)
REFERENCE KANGE(s)
13.3 gnydL
13.0-17.0
RBC
10*12/
4555
HCT
45 %
40-50
MCV
67 fL
80-100
MCH
20 pg
27-32
MCKC
31.5-345
WBC
[0e9,
4.0-10.0
PLATELETS
114 with clumps xl0 9L
150-400
NEUTROFHILS%
51 %
40-S0
LYMPHOCYTES%
38 %
20-40
MONOCYTES%
06 %
2-10
EOSIN...
✅ Found platelets: 114.0
✅ Found hct: 45.0
✅ Found mcv: 67.0
✅ Found mch: 20.0
✅ Found chloride: 0.0
✅ Nearby line - wbc: 9.0
✅ Nearby line - lymphocytes: 38.0
✅ Nearby line - monocytes: 6.0
✅ Nearby line - eosinophils: 2.0
🎯 FINAL EXTRACTION: 9 values found
   PLATELETS: 114.